## Steps for building a new sensor
### A) First step is to define a specific sequence
### B) DIvide the sequence in windows o the same lenght of the trigger ( 36nt)
### C) For each possible trigger, build a sensor 
### D) remove all the ones that have stop codons
### E) evaluate the propierties and rank them low->high errors 

In [16]:
import os
import os.path
from subprocess import Popen, PIPE
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import statsmodels.api as sm
from skbio import Sequence

In [17]:
def reversed_complement(sequence):
    mapping = {'A': 'U', 'G': 'C', 'U': 'A', 'C': 'G'}
    sequence_upper = sequence.upper()

    complement = ''
    for c in sequence_upper:
        complement += mapping[c]

    # reverse the sequence
    return complement[::-1]

In [18]:
def split_sequence(sequence, window):
    sequences = []
    limit = len(sequence) - window + 1

    for i in range(0, limit):
        sequences.append(sequence[i:window + i])

    return sequences

In [19]:
#Starting with a target sequence, in this case is the  5' of the glycoprotein 1 from PVY:
seq= "ATGGCAACTTACATGTCAACAATCTGTTTTGGTTCGTTTGAATGCAAGCTACCATACTCACCAGCCTCTTGCGAGCATATTGTGAAGGAACGAGAAGTGCCGGCTTCCGTTGATCCTTTCGCAGATCTGGAAACACAACTTAGTGCACGATTGCTCAAGCAAAAATATGCTACTGTTCGTGTGCTCAAAAACGGTACTTTTACGTACCGATACAAGACTGATGCCCAGATAATGCGCATTCAGAAGAAACTGGAGAGGAAGGATAGGGAAGAATATCACTTCCAAATGGCCGCTCCTAGTATTGTGTCAAAAATTACTATAGCTGGCGGAGATCCTCCATCAAAGTCTGAGCCACAAGCACCAAGAGGGATCATTCATACAACTCCAAGGATGCGTAAAGTCAAGACACGCCCCATAATAAAGTTGACAGAAGGCCAGATGAATCACCTCATTAAGCAGATAAAACAGATTATGTCGGAGAAAAGAGGGTCTGTCCACTTAATTAGTAAGAAAACCACTCATGTTCAATATAAGAAGATACTTGGTGCATACTCCGCAGCGGTTCGAACTGCACATATGATGGGTTTGCGACGGAGAGTGGACTTCCGATGTGATATGTGGACAGTTGGACTTTTGCAACGTCTCGCTCGGACGGACAAATGGTCCAATCAAGTCCGCACTATCAACATACGAAGGGGTGATAGTGGAGTCATCTTGAACACAAAAAGCCTCAAAGGCCACTTTGGTAGAAGTTCAGGAGGCTTGTTCATAGTGCGTGGATCACACGAAGGGAAATTGTATGATGCACGTTCTAGAGTTACTCAGAGTATTTTAAACTCAATGATCCAGTTTTCGAATGCCGACAATTTTTGGAAGGGTCTGGACGGTAATTGGGCACGAATGAGATATCCTTCGGATCACACATGTGTAGCTGGTTTACCTGTCGAAGATTGTGGTAGGGTAGCTGCATTGATGGCACACAGTATCCTTCCGTGCTATAAGATAACTTGCCCCACCTGTGCTCAACAGTATGCCAGCTTGCCAGTTAGC"
processed_sequence = seq.upper().replace('T', 'U').replace(' ', '')

In [20]:
rc=reversed_complement(processed_sequence)
split_sequence(processed_sequence, 36)
len(split_sequence(rc,36))

1015

In [21]:
#making all possible triggers for the direct and reversed complementary sequence"
d_1 = {'Triggers': split_sequence(processed_sequence,36)}
df_1 = pd.DataFrame(data=d_1)
df_1["Sense"]="Direct"

d_2 = {'Triggers': split_sequence(rc,36)}
df_2 = pd.DataFrame(data=d_2)
df_2["Sense"]="Reversed Complement"
frames = [df_1, df_2]
result = pd.concat(frames)
result;

In [22]:
def no_stop(sequence):
    stop = ['UAA', 'UAG', 'UGA']

    for i in range(0, len(sequence), 3):
        if sequence[i:i + 3] in stop:
            return False

    return True

In [23]:
def possible_toehold_B(reg, rev):
    loop = 'GGACUUUAGAACAGAGGAGAUAAAGAUG'

    #What is the RBS ?
    linker = 'ACCTGGCGGCAGCGCAAGAAGA'#( from Green paper 2019 se, modified with an extra A for creating C site MOCLO)
    #linker = "AACCUGGCGGCAGCGCAAGAAGAUGCGUAAA" 
    toeholds = []

    #for n in ['A', 'G', 'U', 'C']:
    if no_stop(reg[0:11] + linker):
            toeholds.append(rev + loop + reg[0:11] + linker)
            return toeholds
      
    
    toeholds.append("STOP")
    return toeholds

In [24]:
#for each trigger, we will design a toeholdso
toeholds=[]
for i in range(len(result.iloc[:,0])):
    toeholds.append((possible_toehold_B(result.iloc[i,0],reversed_complement(result.iloc[i,0])))[0])
toeholds


    
a=pd.Series(toeholds)
#toeholds[0][0]
#toeholds
#aux
#a[0][0]
result["Toehold Switch"]=a.values
result

,Triggers,Sense,Toehold Switch
0,AUGGCAACUUACAUGUCAACAAUCUGUUUUGGUUCG,Direct,STOP
1,UGGCAACUUACAUGUCAACAAUCUGUUUUGGUUCGU,Direct,ACGAACCAAAACAGAUUGUUGACAUGUAAGUUGCCAGGACUUUAGA...
2,GGCAACUUACAUGUCAACAAUCUGUUUUGGUUCGUU,Direct,AACGAACCAAAACAGAUUGUUGACAUGUAAGUUGCCGGACUUUAGA...
3,GCAACUUACAUGUCAACAAUCUGUUUUGGUUCGUUU,Direct,AAACGAACCAAAACAGAUUGUUGACAUGUAAGUUGCGGACUUUAGA...
4,CAACUUACAUGUCAACAAUCUGUUUUGGUUCGUUUG,Direct,STOP
5,AACUUACAUGUCAACAAUCUGUUUUGGUUCGUUUGA,Direct,UCAAACGAACCAAAACAGAUUGUUGACAUGUAAGUUGGACUUUAGA...
6,ACUUACAUGUCAACAAUCUGUUUUGGUUCGUUUGAA,Direct,UUCAAACGAACCAAAACAGAUUGUUGACAUGUAAGUGGACUUUAGA...
7,CUUACAUGUCAACAAUCUGUUUUGGUUCGUUUGAAU,Direct,AUUCAAACGAACCAAAACAGAUUGUUGACAUGUAAGGGACUUUAGA...
8,UUACAUGUCAACAAUCUGUUUUGGUUCGUUUGAAUG,Direct,CAUUCAAACGAACCAAAACAGAUUGUUGACAUGUAAGGACUUUAGA...
9,UACAUGUCAACAAUCUGUUUUGGUUCGUUUGAAUGC,Direct,GCAUUCAAACGAACCAAAACAGAUUGUUGACAUGUAGGACUUUAGA...


In [25]:
#Removing Toehold Sensors with STOP codons

df = result[result.iloc[:,2] != "STOP"]
df

,Triggers,Sense,Toehold Switch
1,UGGCAACUUACAUGUCAACAAUCUGUUUUGGUUCGU,Direct,ACGAACCAAAACAGAUUGUUGACAUGUAAGUUGCCAGGACUUUAGA...
2,GGCAACUUACAUGUCAACAAUCUGUUUUGGUUCGUU,Direct,AACGAACCAAAACAGAUUGUUGACAUGUAAGUUGCCGGACUUUAGA...
3,GCAACUUACAUGUCAACAAUCUGUUUUGGUUCGUUU,Direct,AAACGAACCAAAACAGAUUGUUGACAUGUAAGUUGCGGACUUUAGA...
5,AACUUACAUGUCAACAAUCUGUUUUGGUUCGUUUGA,Direct,UCAAACGAACCAAAACAGAUUGUUGACAUGUAAGUUGGACUUUAGA...
6,ACUUACAUGUCAACAAUCUGUUUUGGUUCGUUUGAA,Direct,UUCAAACGAACCAAAACAGAUUGUUGACAUGUAAGUGGACUUUAGA...
7,CUUACAUGUCAACAAUCUGUUUUGGUUCGUUUGAAU,Direct,AUUCAAACGAACCAAAACAGAUUGUUGACAUGUAAGGGACUUUAGA...
8,UUACAUGUCAACAAUCUGUUUUGGUUCGUUUGAAUG,Direct,CAUUCAAACGAACCAAAACAGAUUGUUGACAUGUAAGGACUUUAGA...
9,UACAUGUCAACAAUCUGUUUUGGUUCGUUUGAAUGC,Direct,GCAUUCAAACGAACCAAAACAGAUUGUUGACAUGUAGGACUUUAGA...
10,ACAUGUCAACAAUCUGUUUUGGUUCGUUUGAAUGCA,Direct,UGCAUUCAAACGAACCAAAACAGAUUGUUGACAUGUGGACUUUAGA...
11,CAUGUCAACAAUCUGUUUUGGUUCGUUUGAAUGCAA,Direct,UUGCAUUCAAACGAACCAAAACAGAUUGUUGACAUGGGACUUUAGA...


## Scoring functions from Green (2019):
#### Three-parameter fit (R2 = 0.57):
#### Fold change = –71.7 dfull_sensor  – 49.1 dactive_sensor – 22.6 dbinding_site + 54.3

#### Four-parameter fit (R2 = 0.60):
#### Fold change = –93.2 dfull_sensor – 43.3 dactive_sensor – 22.1 dbinding_site – 9.4 dmin_target + 61.3

In [26]:
# this is the structure of the designed optimal interaction betwen a Sensor and its trigger
secondary_2=".......................................(((((............)))))....................................+...................................."

In [27]:
# Calculation of the minimum free energy (MFE) secondary structure of a singular RNA sequence

def DG(sequence,result_path,wait):
    file = open('{}pipo.in'.format(result_path), 'w')
    file.write("{}\n".format(sequence))
    file.close()
    final=[]
    semi_final=[]

    Popen(["mfe -T 29 {}pipo".format(result_path)],shell=True, stdout=PIPE)
    time.sleep(wait)
    lenght=len(sequence)
    with open("{}pipo.mfe".format(result_path)) as res:
        for r in res:
            r = r.strip('\n')
            if not r.startswith('%'):
                r = r.split('\t')
                semi_final.append(r)

    #final.append()                

    return (float(semi_final[2][0]))

    os.remove("{}pipo.mfe".format(result_path,))
    os.remove("{}pipo.in".format(result_path))

In [28]:
secondary_sensor_B = '.........................(((((((((((...(((((............)))))...)))))))))))......................'

In [29]:

def nupack_analysis(sequence, secondary_sensor,  window, sensor_type, result_path):
    list_for_table = []

    processed_sequence = sequence.upper().replace('T', 'U').replace(' ', '')
    reg_sequences = split_sequence(processed_sequence, window)
    rev_comp_sequences = [reversed_complement(s) for s in reg_sequences]

    if sensor_type == 'A':
        target_toehold_map = possible_toehold_A(reg_sequences, rev_comp_sequences)
    else:
        target_toehold_map = possible_toehold_B(reg_sequences, rev_comp_sequences)

    sequence = sequence.upper().replace('T', 'U')
    single_streadness_sequence = single_streadness(sequence, result_path, wait=6)
    for target, toehold in target_toehold_map.items():
        id = sequence.index(target)

        target_defect = sum(single_streadness_sequence[id:id+36])/36
        toehold_defect = sum(single_streadness(toehold, result_path)[0:30])/30
        sensor_defect = complex_defect(toehold, secondary_sensor, result_path)

        score = 5*(1-target_defect) + 4*(1-toehold_defect) + 3*sensor_defect

        list_for_table.append(tuple([target[0:36], toehold, 1-target_defect, 1-toehold_defect, sensor_defect, score]))

    return list_for_table